In [37]:
!pip install --quiet -U langchain langchain-community langchain-aws langchain-core pgvector

import os
import pandas as pd
import numpy as np
import json
import psycopg2
import ast
import pgvector
import math
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
import boto3
from langchain_community.embeddings.bedrock import BedrockEmbeddings
from langchain_aws import BedrockLLM
import time
import logging

print("Imported and installed dependencies!")

Imported and installed dependencies!


In [13]:
import os
import json
import boto3
import pandas as pd
import ast

def get_bedrock_embeddings(input_text, model_id="amazon.titan-embed-text-v2:0", region_name="ca-central-1"):
    """Fetches text embeddings from AWS Bedrock."""
    bedrock = boto3.client(service_name='bedrock-runtime', region_name=region_name)
    
    body = json.dumps({
        "inputText": input_text,
        "dimensions": 1024,
        "normalize": True
    })

    response = bedrock.invoke_model(
        body=body,
        modelId=model_id,
        accept="*/*",
        contentType="application/json"
    )

    response_body = json.loads(response['body'].read())
    return response_body.get('embedding', [])

get_bedrock_embeddings("hello")




[-0.05515049770474434,
 0.04516850784420967,
 0.02720092423260212,
 0.006394712720066309,
 0.04142526164650917,
 -0.0359351672232151,
 0.009233341552317142,
 0.0314432717859745,
 0.017094159498810768,
 0.010793027468025684,
 -0.009794828481972218,
 0.03718291595578194,
 0.007611267734318972,
 -0.05415229871869087,
 -0.011104964651167393,
 -0.03543606773018837,
 -0.014972985722124577,
 -0.035685617476701736,
 0.04766400530934334,
 0.002916612895205617,
 -0.03393876925110817,
 0.016220735386013985,
 0.03393876925110817,
 -0.020587855949997902,
 -0.027700023725628853,
 -0.02607795037329197,
 -0.035186517983675,
 -0.03543606773018837,
 0.00948289129883051,
 -0.05714689567685127,
 0.04741445556282997,
 -0.0070497808046638966,
 -0.008235142566263676,
 -0.01397478673607111,
 0.0160959605127573,
 0.03793156519532204,
 0.019963981583714485,
 -0.02345767803490162,
 -0.012352713383734226,
 -0.01647028513252735,
 0.05440184846520424,
 -0.028199123218655586,
 -0.0061139692552387714,
 0.021086955443

In [75]:
def create_dynamodb_history_table(table_name: str) -> bool:
    """
    Create a DynamoDB table to store the session history if it doesn't already exist.

    Args:
    table_name (str): The name of the DynamoDB table to create.

    Returns:
    None
    
    If the table already exists, this function does nothing. Otherwise, it creates a 
    new table with a key schema based on 'SessionId'.
    """
    # Get the service resource and client.
    dynamodb_resource = boto3.resource("dynamodb")
    dynamodb_client = boto3.client("dynamodb")
    
    # Retrieve the list of tables that currently exist.
    existing_tables = []
    exclusive_start_table_name = None
    
    while True:
        if exclusive_start_table_name:
            response = dynamodb_client.list_tables(ExclusiveStartTableName=exclusive_start_table_name)
        else:
            response = dynamodb_client.list_tables()
        
        existing_tables.extend(response.get('TableNames', []))
        
        if 'LastEvaluatedTableName' in response:
            exclusive_start_table_name = response['LastEvaluatedTableName']
        else:
            break
    
    if table_name not in existing_tables:  # Create a new table if it doesn't exist.
        # Create the DynamoDB table.
        table = dynamodb_resource.create_table(
            TableName=table_name,
            KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
            AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
            BillingMode="PAY_PER_REQUEST",
        )
        
        # Wait until the table exists.
        table.meta.client.get_waiter("table_exists").wait(TableName=table_name)

create_dynamodb_history_table("test")

In [15]:
secrets_manager_client = boto3.client("secretsmanager")
ssm_client = boto3.client("ssm", region_name="ca-central-1")

def get_secret(secret_name, expect_json=True):
    global db_secret
    if db_secret is None:
        try:
            response = secrets_manager_client.get_secret_value(SecretId=secret_name)["SecretString"]
            db_secret = json.loads(response) if expect_json else response
        except json.JSONDecodeError as e:
            logger.error(f"Failed to decode JSON for secret {secret_name}: {e}")
            raise ValueError(f"Secret {secret_name} is not properly formatted as JSON.")
        except Exception as e:
            logger.error(f"Error fetching secret {secret_name}: {e}")
            raise
    return db_secret


def get_parameter(param_name, cached_var):
    """
    Fetch a parameter value from Systems Manager Parameter Store.
    """
    if cached_var is None:
        try:
            response = ssm_client.get_parameter(Name=param_name, WithDecryption=True)
            cached_var = response["Parameter"]["Value"]
        except Exception as e:
            logger.error(f"Error fetching parameter {param_name}: {e}")
            raise
    return cached_var

In [16]:
secrets_raw = secrets_manager_client.get_secret_value(SecretId='virt-care-Database-VCI/credentials/rdsDbCredential')["SecretString"]
secrets = json.loads(secrets_raw)

In [76]:


# Set up basic logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# Environment variables
DB_SECRET_NAME = "virt-care-Database-VCI/credentials/rdsDbCredential" #os.environ["SM_DB_CREDENTIALS"]
REGION = "ca-central-1" #os.environ["REGION"]
RDS_PROXY_ENDPOINT = secrets.get("host") #os.environ["RDS_PROXY_ENDPOINT"]
BEDROCK_LLM_PARAM =  "meta.llama3-8b-instruct-v1:0" #os.environ["BEDROCK_LLM_PARAM"]
EMBEDDING_MODEL_PARAM = "amazon.titan-embed-text-v2:0"#os.environ["EMBEDDING_MODEL_PARAM"]
TABLE_NAME_PARAM = "DynamoDB-Conversation-Table" #os.environ["TABLE_NAME_PARAM"]

# AWS Clients
secrets_manager_client = boto3.client("secretsmanager")
ssm_client = boto3.client("ssm", region_name=REGION)
bedrock_runtime = boto3.client("bedrock-runtime", region_name=REGION)

# Cached resources
connection = None
db_secret = None
BEDROCK_LLM_ID = None
EMBEDDING_MODEL_ID = None
TABLE_NAME = "test"

# Cached embeddings instance
embeddings = None



def initialize_constants():
    global BEDROCK_LLM_ID, EMBEDDING_MODEL_ID, TABLE_NAME, embeddings
    BEDROCK_LLM_ID = get_parameter(BEDROCK_LLM_PARAM, BEDROCK_LLM_ID)
    EMBEDDING_MODEL_ID = get_parameter(EMBEDDING_MODEL_PARAM, EMBEDDING_MODEL_ID)
    TABLE_NAME = get_parameter(TABLE_NAME_PARAM, TABLE_NAME)

    if embeddings is None:
        embeddings = BedrockEmbeddings(
            model_id=EMBEDDING_MODEL_ID,
            client=bedrock_runtime,
            region_name=REGION,
        )
    
    create_dynamodb_history_table(TABLE_NAME)

def connect_to_db():
    global connection
    if connection is None or connection.closed:
        try:
            secret = get_secret(DB_SECRET_NAME)
            connection_params = {
                'dbname': secret["dbname"],
                'user': secret["username"],
                'password': secret["password"],
                'host': RDS_PROXY_ENDPOINT,
                'port': secret["port"]
            }
            connection_string = " ".join([f"{key}={value}" for key, value in connection_params.items()])
            connection = psycopg2.connect(connection_string)
            logger.info("Connected to the database!")
        except Exception as e:
            logger.error(f"Failed to connect to database: {e}")
            if connection:
                connection.rollback()
                connection.close()
            raise
    return connection

In [21]:


def get_system_prompt(case_id):
    connection = connect_to_db()
    if connection is None:
        logger.error("No database connection available.")
        return {
            "statusCode": 500,
            "body": json.dumps("Database connection failed.")
        }
    
    try:
        cur = connection.cursor()
        
        cur.execute("""
            SELECT system_prompt, case_title,case_type, law_type
            FROM "cases"
            WHERE case_id = %s;
        """, (case_id,))

        result = cur.fetchone()
        logger.info(f"Query result: {result}")
        system_prompt = result[0] if result else None

        cur.close()

        if system_prompt:
            logger.info(f"System prompt for simulation_group_id {case_id} found: {system_prompt}")
        else:
            logger.warning(f"No system prompt found for simulation_group_id {case_id}")

        return system_prompt

    except Exception as e:
        logger.error(f"Error fetching system prompt: {e}")
        if cur:
            cur.close()
        connection.rollback()
        return None

In [51]:
!pip install langchain_postgres

In [52]:
import logging
from typing import Optional

import psycopg2
from langchain_aws import BedrockEmbeddings
from langchain_postgres import PGVector

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def get_vectorstore(
    collection_name: str, 
    embeddings: BedrockEmbeddings, 
    dbname: str, 
    user: str, 
    password: str, 
    host: str, 
    port: int
) -> Optional[PGVector]:
    """
    Initialize and return a PGVector instance.
    
    Args:
    collection_name (str): The name of the collection.
    embeddings (BedrockEmbeddings): The embeddings instance.
    dbname (str): The name of the database.
    user (str): The database user.
    password (str): The database password.
    host (str): The database host.
    port (int): The database port.
    
    Returns:
    Optional[PGVector]: The initialized PGVector instance, or None if an error occurred.
    """
    try:
        connection_string = (
            f"postgresql+psycopg://{user}:{password}@{host}:{port}/{dbname}"
        )

        logger.info("Initializing the VectorStore")
        vectorstore = PGVector(
            embeddings=embeddings,
            collection_name=collection_name,
            connection=connection_string,
            use_jsonb=True
        )

        logger.info("VectorStore initialized")
        return vectorstore, connection_string

    except Exception as e:
        logger.error(f"Error initializing vector store: {e}")
        return None

In [56]:
from typing import Dict

from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

def get_vectorstore_retriever(
    llm,
    vectorstore_config_dict: Dict[str, str],
    embeddings#: BedrockEmbeddings
) -> VectorStoreRetriever:
    """
    Retrieve the vectorstore and return the history-aware retriever object.

    Args:
    llm: The language model instance used to generate the response.
    vectorstore_config_dict (Dict[str, str]): The configuration dictionary for the vectorstore, including parameters like collection name, database name, user, password, host, and port.
    embeddings (BedrockEmbeddings): The embeddings instance used to process the documents.

    Returns:
    VectorStoreRetriever: A history-aware retriever instance.
    """
    vectorstore, _ = get_vectorstore(
        collection_name=vectorstore_config_dict['collection_name'],
        embeddings=embeddings,
        dbname=vectorstore_config_dict['dbname'],
        user=vectorstore_config_dict['user'],
        password=vectorstore_config_dict['password'],
        host=vectorstore_config_dict['host'],
        port=int(vectorstore_config_dict['port'])
    )

    retriever = vectorstore.as_retriever()

    # Contextualize question and create history-aware retriever
    contextualize_q_system_prompt = (
        """Given a chat history and the latest user question 
        which might reference context in the chat history, 
        formulate a standalone response which can be understood 
        without the chat history, but may make references to past messages as well.
        Just reformulate it if needed and otherwise return it as is."""
    )
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )

    return history_aware_retriever

In [58]:
case_id = '550e8400-e29b-41d4-a716-546655480010'

In [59]:
logger.info("Retrieving vectorstore config.")
db_secret = get_secret(DB_SECRET_NAME)
vectorstore_config_dict = {
            'collection_name': case_id,
            'dbname': db_secret["dbname"],
            'user': db_secret["username"],
            'password': db_secret["password"],
            'host': RDS_PROXY_ENDPOINT,
            'port': db_secret["port"]
    }

INFO:__main__:Retrieving vectorstore config.


In [71]:
import boto3, re
from langchain_aws import ChatBedrock
from langchain_aws import BedrockLLM
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import DynamoDBChatMessageHistory
from pydantic import BaseModel, Field

class LLM_evaluation(BaseModel):
    response: str = Field(description="Assessment of the student's answer with a follow-up question.")
    verdict: str = Field(description="'True' if the student has properly diagnosed the patient, 'False' otherwise.")


def create_dynamodb_history_table(table_name: str) -> bool:
    """
    Create a DynamoDB table to store the session history if it doesn't already exist.

    Args:
    table_name (str): The name of the DynamoDB table to create.

    Returns:
    None
    
    If the table already exists, this function does nothing. Otherwise, it creates a 
    new table with a key schema based on 'SessionId'.
    """
    # Get the service resource and client.
    dynamodb_resource = boto3.resource("dynamodb")
    dynamodb_client = boto3.client("dynamodb")
    
    # Retrieve the list of tables that currently exist.
    existing_tables = []
    exclusive_start_table_name = None
    
    while True:
        if exclusive_start_table_name:
            response = dynamodb_client.list_tables(ExclusiveStartTableName=exclusive_start_table_name)
        else:
            response = dynamodb_client.list_tables()
        
        existing_tables.extend(response.get('TableNames', []))
        
        if 'LastEvaluatedTableName' in response:
            exclusive_start_table_name = response['LastEvaluatedTableName']
        else:
            break
    
    if table_name not in existing_tables:  # Create a new table if it doesn't exist.
        # Create the DynamoDB table.
        table = dynamodb_resource.create_table(
            TableName=table_name,
            KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
            AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
            BillingMode="PAY_PER_REQUEST",
        )
        
        # Wait until the table exists.
        table.meta.client.get_waiter("table_exists").wait(TableName=table_name)

def get_bedrock_llm(
    bedrock_llm_id: str,
    temperature: float = 0
) -> ChatBedrock:
    """
    Retrieve a Bedrock LLM instance based on the provided model ID.

    Args:
    bedrock_llm_id (str): The unique identifier for the Bedrock LLM model.
    temperature (float, optional): The temperature parameter for the LLM, controlling 
    the randomness of the generated responses. Defaults to 0.

    Returns:
    ChatBedrock: An instance of the Bedrock LLM corresponding to the provided model ID.
    """
    return ChatBedrock(
        model_id=bedrock_llm_id,
        model_kwargs=dict(temperature=temperature),
    )

def get_student_query(raw_query: str) -> str:
    """
    Format the student's raw query into a specific template suitable for processing.

    Args:
    raw_query (str): The raw query input from the student.

    Returns:
    str: The formatted query string ready for further processing.
    """
    student_query = f"""
    user
    {raw_query}
    
    """
    return student_query

def get_initial_student_query(patient_name: str) -> str:
    """
    Generate an initial query for the student to interact with the system. 
    The query asks the student to greet the system and then requests a question related to a specified patient.

    Args:
    patient_name (str): The name of the patient for which the initial question should be generated.

    Returns:
    str: The formatted initial query string for the student.
    """
    student_query = f"""
    user
    Greet me and then ask me a question related to the patient: {patient_name}. 
    """
    return student_query

def get_response(
    query: str,
    llm: ChatBedrock,
    history_aware_retriever,
    table_name: str,
    session_id: str,
    system_prompt: str,
    llm_completion: bool
) -> dict:
    """
    Generates a response to a query using the LLM and a history-aware retriever for context.

    Args:
    query (str): The student's query string for which a response is needed.
    patient_name (str): The specific patient that the student needs to diagnose.
    llm (ChatBedrock): The language model instance used to generate the response.
    history_aware_retriever: The history-aware retriever instance that provides relevant context documents for the query.
    table_name (str): The DynamoDB table name used to store and retrieve the chat history.
    session_id (str): The unique identifier for the chat session to manage history.

    Returns:
    dict: A dictionary containing the generated response and the source documents used in the retrieval.
    """
    
    completion_string = """
                Once I, the pharmacy student, have give you a diagnosis, politely leave the conversation and wish me goodbye.
                Regardless if I have given you the proper diagnosis or not for the patient you are pretending to be, stop talking to me.
                """
    if llm_completion:
        completion_string = """
                Continue this process until you determine that me, the pharmacy student, has properly diagnosed the patient you are pretending to be.
                Once the proper diagnosis is provided, include PROPER DIAGNOSIS ACHIEVED in your response and do not continue the conversation.
                """

    # Create a system prompt for the question answering
    system_prompt = get_system_prompt(case_id)
    
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}\n\nContext: {context}")
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        lambda session_id: DynamoDBChatMessageHistory(
            table_name=table_name, 
            session_id=session_id
        ),
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
    
    # Generate the response until it's not empty
    response = ""
    while not response:
        response = generate_response(
            conversational_rag_chain,
            query,
            session_id
        )
    
    return get_llm_output(response, llm_completion)

def generate_response(conversational_rag_chain: object, query: str, session_id: str) -> str:
    """
    Invokes the RAG chain to generate a response to a given query.

    Args:
    conversational_rag_chain: The Conversational RAG chain object that processes the query and retrieves relevant responses.
    query (str): The input query for which the response is being generated.
    session_id (str): The unique identifier for the current conversation session.

    Returns:
    str: The answer generated by the Conversational RAG chain, based on the input query and session context.
    """
    return conversational_rag_chain.invoke(
        {
            "input": query
        },
        config={
            "configurable": {"session_id": session_id}
        },  # constructs a key "session_id" in `store`.
    )["answer"]

def get_llm_output(response: str, llm_completion: bool) -> dict:
    """
    Processes the response from the LLM to determine if proper diagnosis has been achieved.

    Args:
    response (str): The response generated by the LLM.

    Returns:
    dict: A dictionary containing the processed output from the LLM and a boolean 
    flag indicating whether proper diagnosis has been achieved.
    """

    completion_sentence = " Congratulations! You have provided the proper diagnosis for me, the patient I am pretending to be! Please try other mock patients to continue your diagnosis skills! :)"

    if not llm_completion:
        return dict(
            llm_output=response,
            llm_verdict=False
        )
    
    elif "PROPER DIAGNOSIS ACHIEVED" not in response:
        return dict(
            llm_output=response,
            llm_verdict=False
        )
    
    elif "PROPER DIAGNOSIS ACHIEVED" in response:
        sentences = split_into_sentences(response)
        
        for i in range(len(sentences)):
            
            if "PROPER DIAGNOSIS ACHIEVED" in sentences[i]:
                llm_response=' '.join(sentences[0:i-1])
                
                if sentences[i-1][-1] == '?':
                    return dict(
                        llm_output=llm_response,
                        llm_verdict=False
                    )
                else:
                    return dict(
                        llm_output=llm_response + completion_sentence,
                        llm_verdict=True
                    )

def split_into_sentences(paragraph: str) -> list[str]:
    """
    Splits a given paragraph into individual sentences using a regular expression to detect sentence boundaries.

    Args:
    paragraph (str): The input text paragraph to be split into sentences.

    Returns:
    list: A list of strings, where each string is a sentence from the input paragraph.

    This function uses a regular expression pattern to identify sentence boundaries, such as periods, question marks, 
    or exclamation marks, and avoids splitting on abbreviations (e.g., "Dr." or "U.S.") by handling edge cases. The 
    resulting list contains sentences extracted from the input paragraph.
    """
    # Regular expression pattern
    sentence_endings = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s'
    sentences = re.split(sentence_endings, paragraph)
    return sentences

def update_session_name(table_name: str, session_id: str, bedrock_llm_id: str) -> str:
    """
    Check if both the LLM and the student have exchanged exactly one message each.
    If so, generate and return a session name using the content of the student's first message
    and the LLM's first response. Otherwise, return None.

    Args:
    session_id (str): The unique ID for the session.
    table_name (str): The DynamoDB table name where the conversation history is stored.

    Returns:
    str: The updated session name if conditions are met, otherwise None.
    """
    
    dynamodb_client = boto3.client("dynamodb")
    
    # Retrieve the conversation history from the DynamoDB table
    try:
        response = dynamodb_client.get_item(
            TableName=table_name,
            Key={
                'SessionId': {
                    'S': session_id
                }
            }
        )
    except Exception as e:
        print(f"Error fetching conversation history from DynamoDB: {e}")
        return None

    history = response.get('Item', {}).get('History', {}).get('L', [])



    human_messages = []
    ai_messages = []
    
    # Find the first human and ai messages in the history
    # Check if length of human messages is 2 since the prompt counts as 1
    # Check if length of AI messages is 2 since after first response by student, another response is generated
    for item in history:
        message_type = item.get('M', {}).get('data', {}).get('M', {}).get('type', {}).get('S')
        
        if message_type == 'human':
            human_messages.append(item)
            if len(human_messages) > 2:
                print("More than one student message found; not the first exchange.")
                return None
        
        elif message_type == 'ai':
            ai_messages.append(item)
            if len(ai_messages) > 2:
                print("More than one AI message found; not the first exchange.")
                return None

    if len(human_messages) != 2 or len(ai_messages) != 2:
        print("Not a complete first exchange between the LLM and student.")
        return None
    
    student_message = human_messages[0].get('M', {}).get('data', {}).get('M', {}).get('content', {}).get('S', "")
    llm_message = ai_messages[0].get('M', {}).get('data', {}).get('M', {}).get('content', {}).get('S', "")
    
    llm = BedrockLLM(
                        model_id = bedrock_llm_id
                    )
    
    system_prompt = """
        You are given the first message from an AI and the first message from a student in a conversation. 
        Based on these two messages, come up with a name that describes the conversation. 
        The name should be less than 30 characters. ONLY OUTPUT THE NAME YOU GENERATED. NO OTHER TEXT.
    """
    
    prompt = f"""
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        {system_prompt}
        <|eot_id|>
        <|start_header_id|>AI Message<|end_header_id|>
        {llm_message}
        <|eot_id|>
        <|start_header_id|>Student Message<|end_header_id|>
        {student_message}
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
    """
    
    session_name = llm.invoke(prompt)
    return session_name

In [72]:
# Defining Constants
LLAMA_3_8B = "meta.llama3-8b-instruct-v1:0"
LLAMA_3_70B = "meta.llama3-70b-instruct-v1:0"
MISTRAL_7B = "mistral.mistral-7b-instruct-v0:2"
MISTRAL_LARGE = "mistral.mistral-large-2402-v1:0"
LLAMA_3_1_8B = "meta.llama3-1-8b-instruct-v1:0"
LLAMA_3_1_70B = "meta.llama3-1-70b-instruct-v1:0"

In [78]:
def answer_prompt(user_prompt, number_of_docs):

    # Record the start times
    total_start_time = time.time()
    answer_start_time = time.time()

    # Initialize the Bedrock Embeddings model
    # embeddings = BedrockEmbeddings()
    embedding = get_bedrock_embeddings(user_prompt)

    # docs = get_combined_docs(embedding, number_of_docs)

    # divided_docs = split_docs(docs)
    # print(len(divided_docs["docs"]))

    # documents = format_docs(divided_docs["docs"])

    # Get the LLM we want to invoke
    llm = BedrockLLM(
                        model_id=LLAMA_3_8B
                    )
    

    system_prompt = get_system_prompt(case_id)

    history_aware_retriever = get_vectorstore_retriever(
            llm=llm,
            vectorstore_config_dict=vectorstore_config_dict,
            embeddings=embeddings
        )

    # system_prompt = """You are a helpful assistant to a UBC law student who answers with kindness while being concise,
    #                 so that it is easy to read your responses quickly yet still get valuable information from them. No
    #                 need to be conversational, just skip to talking about the content. Refer to the law student in the second person. 
    #                 You will be provided with context to a legal case the law student is interviewing a client about, and you exist to help provide legal
    #                 context to the law student when they provide you with context on certain client cases, and you should
    #                 provide possible follow-up questions for the law student to ask the client to help progress the case more.
    #                 These are NOT for the client to ask a lawyer; this is to help the law student learn what kind of questions to ask
    #                 their clients, so you should only provide followup questions for the law student to ask the client as if they were a lawyer.
    #                 You may also mention certain legal information and implications the law student may have missed, and mention
    #                 which part of Canadian law it is applicable too if possible or helpful. You are NOT allowed hallucinate,
    #                 informational accuracy is important."""
    # system_prompt = "You are a helpful UBC student advising assistant who answers with kindness while being concise. If the question does not relate to UBC, respond with 'IDK.'"
    # system_prompt = """You are a helpful UBC student advising assistant. 
    #                    Using the documents given to you, consicely answer the user's prompt with kindness. 
    #                    If the question does not relate to UBC, respond with 'IDK.'"""

    if llm.model_id == LLAMA_3_8B or llm.model_id == LLAMA_3_70B or llm.model_id == LLAMA_3_1_8B or llm.model_id == LLAMA_3_1_70B:
        prompt = f"""
                {system_prompt}
                
                User: {user_prompt}
                
                Assistant:
                """
    else:
        prompt = f"""{system_prompt}. Provide your answer as if you are talking to a student.
            Here is the question: {user_prompt}
            """

    print("PROMPT------------------- \n" + prompt)
    # answer = llm.invoke(prompt)

    answer=get_response(
                query=user_prompt,
                llm=llm,
                history_aware_retriever=history_aware_retriever,
                table_name=TABLE_NAME,
                session_id=case_id,
                system_prompt=system_prompt,
                llm_completion=False
            )

    # Record the end time and find duration of answer only
    answer_end_time = time.time()
    answer_duration = answer_end_time - answer_start_time

    # check_docs = check_if_documents_relates(divided_docs["docs"], user_prompt, llm)
    # check_additional_docs = check_if_documents_relates(divided_docs["removed_docs"], user_prompt, llm)

    # Record the end time and find duration of the total time of checking over each document
    total_end_time = time.time()
    total_duration = total_end_time - total_start_time

    return {"answer": answer,
            # "docs": check_docs,
            # "additional_docs": check_additional_docs,
            "answer_time": answer_duration, "total_time": total_duration}

# Neatly prints dictionary returned by answer_prompt
def neat_print(response):
    print(f"Answer: {response['answer']}\n")
    
    # print("Documents:")
    # for doc in response['docs']:
    #     print(f"doc_id: {doc['doc_id']}")
    #     print(f"URL: {doc['url']}")
    #     print(f"Titles: {doc['titles']}")
    #     print(f"Text:\n{doc['text']}")
    #     print(f"Links:{doc['links']}")
    #     print(f"Relevance: {doc['relate']}")
    
    print("_________________________________________________________________________________________________________")
    
    # print("Additional Documents:")
    # for doc in response['additional_docs']:
    #     print(f"doc_id: {doc['doc_id']}")
    #     print(f"URL: {doc['url']}")
    #     print(f"Titles: {doc['titles']}")
    #     print(f"Text:\n{doc['text']}")
    #     print(f"Links:{doc['links']}")
    #     print(f"Relevance: {doc['relate']}")
    
    print(f"answer_time: {response['answer_time']}\n")
    print(f"total_time: {response['total_time']}\n")

response = answer_prompt("My client is going through a divorce, they have three children with their wife and has been married for 40 years. My client wants to find out how to keep the possible financial loss to a minimum.", 5)

neat_print(response)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:root:Connected to the database!
INFO:root:Query result: ('You are a helpful assistant to a UBC law student who answers with kindness while being concise, so that it is easy to read your responses quickly yet still get valuable information from them. No need to be conversational, just skip to talking about the content. Refer to the law student in the second person. You will be provided with context to a legal case the law student is interviewing a client about, and you exist to help provide legal context to the law student when they provide you with context on certain client cases, and you should provide possible follow-up questions for the law student to ask the client to help progress the case more. These are NOT for the client to ask a lawyer; this is to help the law student learn what kind of questions to ask their clients, so you should only provide followup questions for the law stu

PROMPT------------------- 

                You are a helpful assistant to a UBC law student who answers with kindness while being concise, so that it is easy to read your responses quickly yet still get valuable information from them. No need to be conversational, just skip to talking about the content. Refer to the law student in the second person. You will be provided with context to a legal case the law student is interviewing a client about, and you exist to help provide legal context to the law student when they provide you with context on certain client cases, and you should provide possible follow-up questions for the law student to ask the client to help progress the case more. These are NOT for the client to ask a lawyer; this is to help the law student learn what kind of questions to ask their clients, so you should only provide followup questions for the law student to ask the client as if they were a lawyer. You may also mention certain legal information and implications t

AttributeError: 'NoneType' object has no attribute 'embed_query'

In [77]:
print(f"TABLE_NAME: {TABLE_NAME}")

TABLE_NAME: test
